## Part 1: Preprocessing

In [32]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [33]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [34]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [35]:
# Create a list of at least 10 column names to use as X data
columns = ["NumCompaniesWorked", "OverTime", "PercentSalaryHike", "PerformanceRating", "RelationshipSatisfaction", "StockOptionLevel", "TotalWorkingYears", "TrainingTimesLastYear", "WorkLifeBalance", "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion", "YearsWithCurrManager"]
# Create X_df using your selected columns

X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes

NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

In [36]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [37]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
No     837
Yes    339
Name: count, dtype: int64

In [38]:
from sklearn.preprocessing import OneHotEncoder
#Use One Hot Encoder
over_time_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')
over_time_encoder.fit(X_train[['OverTime']])
over_time_train = pd.DataFrame(over_time_encoder.transform(X_train[['OverTime']]), columns=over_time_encoder.get_feature_names_out(),index=X_train.index)
over_time_train

,OverTime_No,OverTime_Yes
1097,1.0,0.0
727,1.0,0.0
254,1.0,0.0
1175,1.0,0.0
1341,1.0,0.0
...,...,...
1130,1.0,0.0
1294,1.0,0.0
860,0.0,1.0
1459,0.0,1.0


In [39]:
over_time_test = pd.DataFrame(over_time_encoder.transform(X_test[['OverTime']]), columns=over_time_encoder.get_feature_names_out(),index=X_test.index)
over_time_test

,OverTime_No,OverTime_Yes
1041,1.0,0.0
184,1.0,0.0
1222,1.0,0.0
67,1.0,0.0
220,1.0,0.0
...,...,...
567,1.0,0.0
560,1.0,0.0
945,0.0,1.0
522,1.0,0.0


In [40]:
#Merge DFs drop Overtime columns from X_test X_train
X_train = X_train.drop(columns='OverTime').join(over_time_train)
X_test = X_test.drop(columns='OverTime').join(over_time_test)

In [41]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_trained_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_one_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
department_one_encoder.fit(np.array(y_train[['Department']]).reshape(-1,1))
y_department_train = department_one_encoder.transform(np.array(y_train[['Department']]).reshape(-1,1))

y_department_test = department_one_encoder.transform(np.array(y_test[['Department']]).reshape(-1,1))
# Create two new variables by applying the encoder
# to the training and testing data
y_department_test

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0

In [61]:
# Create a OneHotEncoder for the Attrition column

attrition_one_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_one_encoder.fit(np.array(y_train[['Attrition']]).reshape(-1,1))
y_attrition_train = attrition_one_encoder.transform(np.array(y_train[['Attrition']]).reshape(-1,1))

y_attrition_test = attrition_one_encoder.transform(np.array(y_test[['Attrition']]).reshape(-1,1))
# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train.shape[1]

2

## Part 2: Create, Compile, and Train the Model

In [50]:
# Find the number of columns in the X training data.
X_columns = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(X_columns,), name='input_layer')


# Create at least two shared layers

shared_layer_1 = layers.Dense(64, activation='relu', name="shared_layer_1")(input_layer)
shared_layer_2 = layers.Dense(128, activation='relu', name="shared_layer_2")(shared_layer_1)

In [67]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu', name="department_hidden_layer")(shared_layer_2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1], activation='softmax', name="department_output")(department_hidden_layer)

In [68]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu', name="attrition_hidden_layer")(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1], activation='softmax', name="attrition_output")(attrition_hidden_layer)

In [69]:
# Create the model
model = Model(inputs=input_layer, outputs={
    "department_output": department_output,
    "attrition_output": attrition_output
})

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_1      │ (None, 64)        │        960 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_2      │ (None, 128)       │      8,320 │ shared_layer_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden_l… │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden_… │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,701 (69.14 KB)

 Trainable params: 17,701 (69.14 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
# Train the model
model.fit(
    X_trained_scaled,
    {
        'department_output': y_department_train,
        'attrition_output': y_attrition_train
    },
    epochs=100
)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - attrition_output_accuracy: 0.7888 - attrition_output_loss: 0.5223 - department_output_accuracy: 0.5520 - department_output_loss: 0.9382 - loss: 1.4606
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8421 - attrition_output_loss: 0.3749 - department_output_accuracy: 0.6722 - department_output_loss: 0.7270 - loss: 1.1018
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8829 - attrition_output_loss: 0.3206 - department_output_accuracy: 0.7026 - department_output_loss: 0.6711 - loss: 0.9917
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8699 - attrition_output_loss: 0.3115 - department_output_accuracy: 0.7434 - department_output_loss: 0.6354 - loss: 0.9468
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8731 - attrition_output_loss: 0.3117 - department_output_accuracy: 0.7479 - department_output_loss: 0.

In [74]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {
        'department_output': y_department_test,
        'attrition_output': y_attrition_test
})
print(results)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - attrition_output_accuracy: 0.7973 - attrition_output_loss: 1.6420 - department_output_accuracy: 0.4756 - department_output_loss: 3.9564 - loss: 5.5991
[5.162653923034668, 3.6966729164123535, 1.462491750717163, 0.8163265585899353, 0.49659863114356995]


In [73]:
# Print the accuracy for both department and attrition
print(f"Attrition predictions accuracy: {results[3]}")
print(f"Department predictions accuracy: {results[4]}")

Attrition predictions accuracy: 0.8163265585899353
Department predictions accuracy: 0.49659863114356995


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Not Necessarily, If the attrition data set is imnbalanced, accuracy may be misleading. Metrics like precision & recall could be better for evaluating performance on inbalanced binary classificaiton
2. For my output layers I used softmax because it is better for multi-class classification, and because they are mutually exclusive
3. Try more layers with different numbers of nuerons in the hidden layers, add more columns, Do hyperparameter tuning and or feature engineering.